In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
pd.reset_option('max_columns')
df_original = pd.read_csv('../data/raw/OnlineRetail.csv', encoding='latin-1')
df_original.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


###Data Cleaning
1.  Removing Nulls

In [66]:
df_original.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [67]:
# delete rows with missing data
df_without_null = df_original.dropna()

In [68]:
#Verification
df_without_null.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

2. Review Negative Data

In [69]:
# negative value
df_without_null[df_without_null['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
141,C536379,D,Discount,-1,12/1/2010 9:41,27.50,14527.0,United Kingdom
154,C536383,35004C,SET OF 3 COLOURED FLYING DUCKS,-1,12/1/2010 9:49,4.65,15311.0,United Kingdom
235,C536391,22556,PLASTERS IN TIN CIRCUS PARADE,-12,12/1/2010 10:24,1.65,17548.0,United Kingdom
236,C536391,21984,PACK OF 12 PINK PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
237,C536391,21983,PACK OF 12 BLUE PAISLEY TISSUES,-24,12/1/2010 10:24,0.29,17548.0,United Kingdom
...,...,...,...,...,...,...,...,...
540449,C581490,23144,ZINC T-LIGHT HOLDER STARS SMALL,-11,12/9/2011 9:57,0.83,14397.0,United Kingdom
541541,C581499,M,Manual,-1,12/9/2011 10:28,224.69,15498.0,United Kingdom
541715,C581568,21258,VICTORIAN SEWING BOX LARGE,-5,12/9/2011 11:57,10.95,15311.0,United Kingdom
541716,C581569,84978,HANGING HEART JAR T-LIGHT HOLDER,-1,12/9/2011 11:58,1.25,17315.0,United Kingdom


In [70]:
# see only the positive value
df_positive_qty = df_without_null[df_without_null['Quantity'] >= 0]

In [71]:
#Validation
df_positive_qty[df_positive_qty['Quantity'] < 0]

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country


3. Duplicate Data

In [72]:
df_positive_qty.duplicated().sum()

np.int64(5192)

In [73]:
# permanently delete duplicate data
df_positive_qty.drop_duplicates(inplace=True)

/var/folders/l4/47d72kkx3dn3rcmtqd6gykbc0000gn/T/ipykernel_2534/3234945824.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_positive_qty.drop_duplicates(inplace=True)


In [74]:
#Verification
df_positive_qty.duplicated().sum()

np.int64(0)

4. Review Stock Code Column ???

In [75]:
# clean the stockcode column data
#a 5-digit integral number uniquely assigned to each distinct product
df_sc_clean = df_positive_qty[df_positive_qty['StockCode'].str.match(r'^\d{5}$')]

In [76]:
df_sc_clean.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom


### EDA

In [77]:
df_sc_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 358309 entries, 1 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    358309 non-null  object 
 1   StockCode    358309 non-null  object 
 2   Description  358309 non-null  object 
 3   Quantity     358309 non-null  int64  
 4   InvoiceDate  358309 non-null  object 
 5   UnitPrice    358309 non-null  float64
 6   CustomerID   358309 non-null  float64
 7   Country      358309 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 24.6+ MB


In [78]:
df_sc_clean.describe()

,Quantity,UnitPrice,CustomerID
count,358309.000000,358309.000000,358309.000000
mean,13.264978,2.892382,15289.146656
std,189.767957,4.382899,1712.352686
min,1.000000,0.000000,12346.000000
25%,2.000000,1.250000,13956.000000
50%,6.000000,1.790000,15152.000000
75%,12.000000,3.750000,16791.000000
max,80995.000000,649.500000,18287.000000


In [79]:
# quantity sort
df_sc_clean.sort_values('Quantity')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
468861,576559,23143,ZINC WIRE KITCHEN ORGANISER,1,11/15/2011 13:19,10.40,14711.0,United Kingdom
116242,546230,22423,REGENCY CAKESTAND 3 TIER,1,3/10/2011 12:26,12.75,13871.0,United Kingdom
241473,558232,22483,RED GINGHAM TEDDY BEAR,1,6/27/2011 14:30,2.95,17841.0,United Kingdom
462337,575972,22694,WICKER STAR,1,11/13/2011 13:01,2.10,14499.0,United Kingdom
462340,575972,22562,MONSTERS STENCIL CRAFT,1,11/13/2011 13:01,1.25,14499.0,United Kingdom
...,...,...,...,...,...,...,...,...
206121,554868,22197,SMALL POPCORN HOLDER,4300,5/27/2011 10:52,0.72,13135.0,United Kingdom
421632,573008,84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,4800,10/27/2011 12:26,0.21,12901.0,United Kingdom
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,11/25/2011 15:57,0.00,13256.0,United Kingdom
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom


In [80]:
# create a new column
df_sc_clean['Total Sales'] = df_sc_clean['Quantity'] * df_sc_clean['UnitPrice']


/var/folders/l4/47d72kkx3dn3rcmtqd6gykbc0000gn/T/ipykernel_2534/4163572493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sc_clean['Total Sales'] = df_sc_clean['Quantity'] * df_sc_clean['UnitPrice']


In [81]:
df_sc_clean.sort_values('Total Sales')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Sales
130188,547417,22062,CERAMIC BOWL WITH LOVE HEART DESIGN,36,3/23/2011 10:25,0.00,13239.0,United Kingdom,0.00
480649,577314,23407,SET OF 2 TRAYS HOME SWEET HOME,2,11/18/2011 13:23,0.00,12444.0,Norway,0.00
454464,575579,22089,PAPER BUNTING VINTAGE PAISLEY,24,11/10/2011 11:49,0.00,13081.0,United Kingdom,0.00
436597,574175,22065,CHRISTMAS PUDDING TRINKET POT,12,11/3/2011 11:47,0.00,14110.0,United Kingdom,0.00
40089,539722,22423,REGENCY CAKESTAND 3 TIER,10,12/21/2010 13:45,0.00,14911.0,EIRE,0.00
...,...,...,...,...,...,...,...,...,...
52711,540815,21108,FAIRY CAKE FLANNEL ASSORTED COLOUR,3114,1/11/2011 12:55,2.10,15749.0,United Kingdom,6539.40
348325,567423,23243,SET OF TEA COFFEE SUGAR TINS PANTRY,1412,9/20/2011 11:05,5.06,17450.0,United Kingdom,7144.72
222680,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,6/10/2011 15:28,649.50,15098.0,United Kingdom,38970.00
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,1/18/2011 10:01,1.04,12346.0,United Kingdom,77183.60


In [82]:
# Customer ID with the most frequent transactions
df_sc_clean['CustomerID'].value_counts()

CustomerID
17841.0    6940
14911.0    5104
14096.0    4558
12748.0    3969
14606.0    2328
           ... 
16073.0       1
13106.0       1
17347.0       1
17941.0       1
15070.0       1
Name: count, Length: 4315, dtype: int64

In [83]:
# countries with the most transactions
df_sc_clean['Country'].value_counts()

Country
United Kingdom          318847
Germany                   8093
France                    7641
EIRE                      6552
Netherlands               2225
Spain                     2209
Belgium                   1798
Switzerland               1669
Portugal                  1339
Australia                 1120
Norway                     978
Channel Islands            704
Italy                      695
Cyprus                     561
Finland                    533
Sweden                     409
Austria                    359
Denmark                    337
Japan                      305
Poland                     300
Israel                     221
Unspecified                218
Singapore                  199
USA                        167
Iceland                    162
Greece                     135
Canada                     126
Malta                       89
United Arab Emirates        66
RSA                         54
European Community          53
Lebanon                     43


In [84]:
# look at the invoice column with the prefix c
#df_sc_clean[df_sc_clean['InvoiceNo'].str.startswith('C')]

In [85]:
list(df_sc_clean.columns)

['InvoiceNo',
 'StockCode',
 'Description',
 'Quantity',
 'InvoiceDate',
 'UnitPrice',
 'CustomerID',
 'Country',
 'Total Sales']

In [86]:
df_sc_clean['InvoiceNo'].str.startswith('C')

1         False
5         False
6         False
7         False
8         False
          ...  
541904    False
541905    False
541906    False
541907    False
541908    False
Name: InvoiceNo, Length: 358309, dtype: bool

In [87]:
df_sc_clean.sort_values('InvoiceNo')

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Total Sales
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom,15.30
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom,25.50
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom,11.10
...,...,...,...,...,...,...,...,...,...
541896,581587,22555,PLASTERS IN TIN STRONGMAN,12,12/9/2011 12:50,1.65,12680.0,France,19.80
541895,581587,22556,PLASTERS IN TIN CIRCUS PARADE,12,12/9/2011 12:50,1.65,12680.0,France,19.80
541894,581587,22631,CIRCUS PARADE LUNCH BOX,12,12/9/2011 12:50,1.95,12680.0,France,23.40
541900,581587,22730,ALARM CLOCK BAKELIKE IVORY,4,12/9/2011 12:50,3.75,12680.0,France,15.00
